# Adaptive PDE discretizations on cartesian grids 
## Volume : GPU accelerated methods
## Part : Reproducibility
## Chapter : Riemannian metrics

In this notebook, we solve isotropic eikonal equations on the CPU and the GPU, and check that they produce consistent results.

In [5]:
import sys; sys.path.insert(0,"../..")
#from Miscellaneous import TocTools; print(TocTools.displayTOC('Isotropic_Repro','GPU'))

In [6]:
import cupy as cp
import numpy as np
import itertools
from matplotlib import pyplot as plt
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%5.3g" % x))

In [26]:
from agd import HFMUtils
from agd import AutomaticDifferentiation as ad
from agd import Metrics
import agd.AutomaticDifferentiation.cupy_generic as cugen
norm_infinity = ad.Optimization.norm_infinity
from agd.HFMUtils import RunGPU,RunSmart

In [28]:
def ReloadPackages():
    from Miscellaneous.rreload import rreload
    global HFMUtils,ad,cugen,RunGPU,Metrics
    HFMUtils,ad,cugen,RunGPU,Metrics = rreload([HFMUtils,ad,cugen,RunGPU,Metrics],"../..")    

In [29]:
cp = cugen.decorate_module_functions(cp,cugen.set_output_dtype32) # Use float32 and int32 types in place of float64 and int64
plt = cugen.decorate_module_functions(plt,cugen.cupy_get_args)
RunSmart = cugen.cupy_get_args(RunSmart,dtype64=True,iterables=(dict,))

In [30]:
from Notebooks_GPU.ExportedCode.Isotropic_Repro import RunCompare

## 1. Two dimensions

In [46]:
ReloadPackages()

In [47]:
n=200
hfmIn = HFMUtils.dictIn({
    'model':'Riemann2',
    'metric':Metrics.Riemann(cp.eye(2)),
    'seeds':[[0,0]],
    'exportValues':1,
})
hfmIn.SetRect([[0,1],[0,1]],dimx=n)

Casting output of function eye from float64 to float32


In [48]:
RunGPU(hfmIn)

Setting the kernel traits.
Prepating the domain data (shape,metric,...)
Preparing the values array (setting seeds,...)
Preparing the GPU kernel
Setup and run the eikonal solver
geom <class 'cupy.core.core.ndarray'>
values <class 'numpy.ndarray'>
seedTags <class 'numpy.ndarray'>


TypeError: Unsupported type <class 'numpy.ndarray'>

In [ ]:
np.isscalar(1.)